# LRE on Hardware

## Executor and Backend Setup

First import all required libraries and modules.

In [17]:
import json
import pandas as pd
import os
import time
import matplotlib.pyplot as plt

from functools import partial
import qiskit
from qiskit_aer import QasmSimulator
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler, QiskitRuntimeService
from qiskit_aer.noise import NoiseModel

from mitiq import lre
from mitiq.interface.mitiq_qiskit.qiskit_utils import initialized_depolarizing_noise
from mitiq.lre.multivariate_scaling.layerwise_folding import _get_num_layers_without_measurements

Define an executor 

In [2]:
def ibmq_executor(circuit: qiskit.QuantumCircuit, backend, shots: int = 100) -> float:
    """Returns the expectation value to be mitigated.

    Args:
        circuit: Circuit to run (assumes no measurements).
        shots: Number of times to execute the circuit to compute the expectation value.
    """
    # Transpile the circuit so it can be properly run
    exec_circuit = qiskit.transpile(
        circuit,
        backend=backend,
        basis_gates=None,
        optimization_level=0,  # Important to preserve folded gates.
    )
    exec_circuit.measure_active()

    num_qubits = circuit.num_qubits

    # Run the circuit
    sampler = Sampler(mode=backend)
    job = sampler.run([exec_circuit], shots=shots)

    # Convert from raw measurement counts to the expectation value
    counts = job.result()[0].data.measure.get_counts()
    # print(f"{exec_circuit.num_qubits} - {num_qubits} - {counts}")
    ground_state = "0" * num_qubits
    expectation_value = counts.get(ground_state, 0.0) / shots
    return expectation_value

In [3]:
# HARDWARE
service = QiskitRuntimeService()
# hard_backend = service.least_busy(operational=True, simulator=False)
hard_backend = service.backend("ibm_sherbrooke")
print(hard_backend)

hardware_executor = partial(ibmq_executor, backend=hard_backend)




# SIMULATOR
noise_model = NoiseModel.from_backend(hard_backend)
sim_backend = AerSimulator.from_backend(hard_backend)

# old_sim_backend = QasmSimulator(method="density_matrix", noise_model=noise_model)
# new_sim_backend = AerSimulator(method="density_matrix", noise_model=noise_model)

sim_backend_ideal = QasmSimulator()

simulator_executor = partial(ibmq_executor, backend=sim_backend)

<IBMBackend('ibm_sherbrooke')>


## Hyperparameter Tuning

First step is to do tests on simulators for hyperparamter tuning. With LRE, the primary paramters that can be adjusted are `degree` and `fold_multiplier`. Due to performance, it has been noted that `num_chunks` will also be critical for testing.

### Manual Example for testing

In [22]:
num_qubits = 4 # 9, 16, 25, 36, 49 (available through benchpress)
qasm_filename = f"square-heisenberg-{num_qubits}"
# qasm_filename = f"qft-{num_qubits}"
circuit = qiskit.qasm2.load(f"{qasm_filename}.qasm")

CIRCUIT_NAME = qasm_filename
DEGREE = 3
FOLD_MULTIPLIER = 2
NUM_CHUNKS = 0

ideal = ibmq_executor(circuit, sim_backend_ideal)
unmitigated = ibmq_executor(circuit, sim_backend)
# new_unmitigated = ibmq_executor(circuit, new_sim_backend)
# old_unmitigated = ibmq_executor(circuit, old_sim_backend)



mitigated = lre.execute_with_lre(
    circuit, simulator_executor, degree=2, fold_multiplier=2, num_chunks=2
)


print(f"ideal={ideal:.3f} unmitigated={unmitigated:.3f} mitigiated={mitigated:.3f}")
# no chunks took > 7 minutes

ideal=1.000 unmitigated=0.590 mitigiated=0.727


In [35]:
print(circuit)
print(len(circuit))

                        ┌─────────┐                   ┌──────────┐┌─────────┐»
q_0: ──■─────────────■──┤ Rx(π/2) ├──■─────────────■──┤ Rx(-π/2) ├┤ Ry(π/2) ├»
       │             │  └─────────┘  │             │  └──────────┘└─────────┘»
q_1: ──┼─────────────┼───────────────┼─────────────┼─────────────────────────»
     ┌─┴─┐┌───────┐┌─┴─┐┌─────────┐┌─┴─┐┌───────┐┌─┴─┐┌──────────┐┌─────────┐»
q_2: ┤ X ├┤ Rz(1) ├┤ X ├┤ Rx(π/2) ├┤ X ├┤ Rz(1) ├┤ X ├┤ Rx(-π/2) ├┤ Ry(π/2) ├»
     └───┘└───────┘└───┘└─────────┘└───┘└───────┘└───┘└──────────┘└─────────┘»
q_3: ────────────────────────────────────────────────────────────────────────»
                                                                             »
«     ┌───────┐                   ┌─────────┐┌───────┐                   »
«q_0: ┤ Rx(π) ├──■─────────────■──┤ Ry(π/2) ├┤ Rx(π) ├──■─────────────■──»
«     └───────┘  │             │  └─────────┘└───────┘  │             │  »
«q_1: ───────────┼─────────────┼────────────────────────┼───────

### Automated Example with Saving Results

First setup the parameters to test including: degree, fold_multiplier, and num_chunks.

In [10]:
MAX_DEGREE = 10
MAX_FOLD_MULTIPLIER = 4 
MAX_NUM_CHUNKS = 5 

# num_qubits = [4, 9, 16, 25, 36, 49]
num_qubits = [4]
# qasm_filenames = [f"square-heisenberg-{num_qubit}" for num_qubit in num_qubits]
qasm_filenames = [f"qft-{num_qubit}" for num_qubit in num_qubits]

# Setup all of the different values/circuits to test
circuits = [qiskit.qasm2.load(f"{qasm_filename}.qasm") for qasm_filename in qasm_filenames]
degrees = list(range(1,MAX_DEGREE+1))
fold_multipliers = list(range(1, MAX_FOLD_MULTIPLIER+1))
num_chunks = list(range(1, MAX_NUM_CHUNKS+1))


Run experiments on a noisy simulator to get results and store them into a results/cached_results.csv for later analysis.

In [11]:
for i in range(len(circuits)):
    circuit = circuits[i]
    for fold_multiplier in fold_multipliers:
        for degree in degrees:
            for num_chunk in num_chunks:
                if num_chunk >= len(circuit):
                    break
                t0 = time.time()
                try:
                    res = lre.execute_with_lre(circuit, simulator_executor, degree=degree, fold_multiplier=fold_multiplier, num_chunks=num_chunk)
                except RuntimeWarning as w:
                    print(f"degree={degree} fold_multiplier={fold_multiplier} num_chunks={num_chunks}: {w}")
                    break
                except ValueError as e:
                    if "Determinant of sample matrix cannot be calculated as the matrix is too large" in str(e):
                        break  # Go to the next iteration of the loop
                    else:  # Re-raise if it's a different ValueError
                        raise  # Important: Don't silently ignore other ValueErrors!
                except Exception as e: #Catch other exceptions
                    raise #Reraise the exception

                t0 = time.time()-t0
                cache_res = [
                   qasm_filenames[i],
                   sim_backend.name,
                   res,
                   degree,
                   fold_multiplier,
                   num_chunk,
                   t0
                ]

                # if backend.name != "qasm_simulator":
                file_name = 'results/cached_results.csv'
                if not os.path.exists(file_name):
                    df = pd.DataFrame(columns=['circuit', 'backend', 'result', 'degree', 'fold_multiplier', 'num_chunks', 'time'])
                    df.to_csv(file_name, index=False)
                df = pd.read_csv(file_name)
                # if sim_backend.name == "qasm_simulator": # qasm_simulator is used for ideal and this only needs to be cached once
                #     existing = df.loc[(df['circuit'] == qasm_filenames[i]) & (df['backend'] == "qasm_simulator")]
                #     if len(existing) >= 1:
                #         return expectation_value

                df.loc[len(df)] = cache_res
                df.sort_values(by=['circuit', 'backend'])
                df.to_csv(file_name, index=False)

/Users/briangoldsmith/anaconda3/envs/mitiqMain/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiqMain/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiqMain/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiqMain/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangoldsmith/anaconda3/envs/mitiqMain/lib/python3.12/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/briangol

In [32]:
# Read from CSV results
file_name = 'results/cached_results.csv'
df = pd.read_csv(file_name)
df.sort_values(by=["circuit, result"])
squareh_4 = df.loc[(df["circuit"] == "square-heisenberg-4") & (df["backend"] == "aer_simulator_from(ibm_sherbrooke)")]
degrees = squareh_4["degree"]
fold_multipliers = squareh_4["fold_multiplier"]
num_chunks = squareh_4["num_chunks"]
results = squareh_4["result"]

plt.figure(figsize=(8,6))
scatter = plt.scatter(degrees, fold_multipliers, c=num_chunks, cmap="viridis", s = (1-results)*100)

plt.xlabel("degree")
plt.ylabel("fold_multiplier")
plt.title("Hyperparameter tuning for LRE for square-heisenberg-4")
plt.colorbar(scatter, label="num_chunks")

plt.grid(True)
plt.tight_layout()
plt.show()

SyntaxError: unterminated string literal (detected at line 4) (524219986.py, line 4)

In [33]:
squareh_4.loc[squareh_4["result"] = ideal - squareh_4["result"]

/var/folders/sk/07gyh_wd1cb_h1jzc9mntfjr0000gn/T/ipykernel_11669/4145829816.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  squareh_4["result"] = ideal - squareh_4["result"]


In [29]:
squareh_4

,circuit,backend,result,degree,fold_multiplier,num_chunks,time
0,square-heisenberg-4,aer_simulator_from(ibm_sherbrooke),0.800000,1,1,1,3.502733
1,square-heisenberg-4,aer_simulator_from(ibm_sherbrooke),0.780000,1,1,2,4.610978
2,square-heisenberg-4,aer_simulator_from(ibm_sherbrooke),0.765000,1,1,3,5.866626
3,square-heisenberg-4,aer_simulator_from(ibm_sherbrooke),0.705000,1,1,4,7.310444
4,square-heisenberg-4,aer_simulator_from(ibm_sherbrooke),0.750000,1,1,5,8.326325
...,...,...,...,...,...,...,...
410,square-heisenberg-4,aer_simulator_from(ibm_sherbrooke),0.588842,7,10,1,38.780741
411,square-heisenberg-4,aer_simulator_from(ibm_sherbrooke),0.621321,7,10,2,129.705728
412,square-heisenberg-4,aer_simulator_from(ibm_sherbrooke),0.738364,8,10,1,50.087394
413,square-heisenberg-4,aer_simulator_from(ibm_sherbrooke),0.603555,9,10,1,62.065930
